Code to clean the spots: retain only the spots which are on the "fish area", ie in a neighboorhood of the cells. 
If you have two genes, run this notebook twice.

23/04/25     Jacques Bourg @ Florian Muller lab. Institut Pasteur.

<img src="./HOX_pipeline.png" alt="LNP pipeline" width="1200" height="483">

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
import skimage.io as io
import napari
import ipywidgets as widgets
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
sys.path.append(str(base_dir / "segmentation"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.file_handling import FileProcessor
from segmentation.refine_seg import Segmentation

tk  = Track()
sg  = Segmentation()
fp  = FileProcessor()

In [ ]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

In [ ]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0] 
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

In [ ]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

In [ ]:
n2  = np.where(np.array(modalities) == dropdown2.value)[0][0] 
modality = modalities[n2]; print(modality)

In [ ]:
channel_cell = constants['CHANNELS'] # DAPI doesn't make sense
dropdown3 = widgets.Dropdown(options=channel_cell, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown3)

In [ ]:
n3     = np.where(np.array(channel_cell) == dropdown3.value)[0][0]
chan_c = channel_cell[n3]; print(chan_c)

In [ ]:
methods = ['UFISH','BIGFISH']  
dropdown4 = widgets.Dropdown(options=methods, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown4)

In [ ]:
n4   = np.where(np.array(methods) == dropdown4.value)[0][0]
meth = methods[n4]; print(meth)

In [ ]:
batch_mod_fish_mip   = constants[f'BATCH_{modality}_DAPI_NUCLEI_MIP']
mask_fish_area_path  = constants[f'MASK_{modality}_FISH_AREA_PATH']
spots_folder         = constants[f'SPOTS_FOLDER_{modality}_{chan_c}_PATH']
dots_path            = constants[f'DOTS_{meth}_{modality}_{chan_c}_PATH']

In [ ]:
spots_dict     = fp.load_spots_distributed_files(dots_path, constants['SPOTS_FORMAT'])
fish_area_dict = fp.load_masks_distributed_files(mask_fish_area_path)

In [ ]:
viewer8  = napari.Viewer(title="FISH Spots cleaning")
counter  = 0

for ind, file in enumerate(batch_mod_fish_mip):

    file                  = Path(file)
    file_name             = str(file.stem)
    base_name             = '_'.join(Path(file).stem.split('_')[:-3])
    
    df_spots              = spots_dict[base_name]
    mask_fish             = fish_area_dict[base_name]
     
    df_spots              = sg.add_column_in_mask_fish(df_spots, mask_fish)
    spots_dict[base_name] = df_spots 

    spots_raw_2d          = df_spots[['Y','X']].to_numpy()
    spots_clean_2d        = df_spots[df_spots['in_mask'] == 1][['Y', 'X']].to_numpy()
        
    im                    = io.imread(file)    
    viewer8.add_image(im, contrast_limits=[0, np.percentile(im, 99)], rgb=False, name=f"FISH {file_name}", opacity=0.8)
    viewer8.add_labels(mask_fish, name=f"FISH AREA {file_name}", opacity=.3, blending='translucent')
    viewer8.add_points(spots_raw_2d, name=f"Spots  {file_name}", size=4, face_color='red')
    viewer8.add_points(spots_clean_2d, name=f"Spots cleaned{file_name}", size=7, border_color='#00FF00', face_color=[0, 0, 0, 0])
 
    if counter != 0:
        viewer8.layers[f"FISH {file_name}"].visible   = False
        viewer8.layers[f"FISH AREA {file_name}"].visible = False
        viewer8.layers[f"Spots  {file_name}"].visible = False 
        viewer8.layers[f"Spots cleaned{file_name}"].visible = False 
    
    counter = counter + 1
 
fp.save_spots_distributed_files(dots_path, constants['SPOTS_FORMAT'], spots_dict, im.ndim)

##### Save parameters

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))